## Generate FSM data
- BehavePlus v.6 (BP6)
- FBP
- KITRAL

In [1]:
import os, itertools, subprocess
import pandas as pd
import numpy as np

In [2]:
os.chdir('/Users/minho/Documents/Github/Cell2Fire/notebooks')

#### BP6

In [ ]:
# Import csv file with input features to calculate Behave Plus
# df = pd.read_csv('/Users/minho/Desktop/Cell2Fire/behaveplus/behaveplus_csv.csv')

# Iterate through different dataset combinations to create csv to push into Cell2FireC
# fuel model (40) x ws (6) x wd (4) x scenario (4) x slope (6)
a = [[101, 102, 103, 104, 105, 106, 107, 108, 109, 121, 122, 123, 124, 141, 142, 143, 144, 145, 146, 147, 148, 149, 161, 162, 163, 164, 165, 181, 182, 183, 184, 185, 186, 187, 188, 189,201, 202, 203, 204], 
     [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90], # WS
     # [0, 90, 180, 270], # WD
     [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85], # Slope
     [1,2,3,4]]

aa = list(itertools.product(*a))

adf = pd.DataFrame(np.asarray(aa), columns = ['nftype','ws','slope','scenario'])

# Moisture Content
mc_dict = {1: [3, 4, 5, 30, 60],
           2: [6, 7, 8, 60, 90],
           3: [9, 10, 11, 90, 120],
           4: [12, 13, 14, 120, 150]}

mc = pd.DataFrame(adf['scenario'].map(mc_dict).tolist(), columns = ['mc1','mc10','mc100','mcWoody','mcHerb'] )

# Merge df together
adf = adf.drop('scenario', axis=1)
adf = pd.concat([adf, mc], axis=1)

# Save
# adf.to_csv('behaveplus_training_data.csv')

#### FBP (CANADA)

In [3]:
# Data generator
def data_generator(FT=['C1','C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'D1', 'D2', 'S1', 'S2', 'S3', 'O1a', 'O1b', 'M1', 'M2', 'M3', 'M4'],
                   WS=[0,100,1],
                   FFMC=[0,101,1],
                   BUI=[0,99,1],
                   ps=[0,100,1], 
                   saz=[0,360,1], 
                   specific_values=False):
    
    # Constants
    # Lat/lon (Arbitrary for now)
    lat = 51.621244
    lon = -115.608378
    time = 20
    
    # GFL dictionary
    GFLD = {"C1": 0.75, "C2": 0.8, "C3": 1.15, "C4": 1.2, "C5":1.2, "C6":1.2, "C7":1.2, 
            "D1": np.nan, "D2": np.nan, 
            "S1":np.nan, "S2": np.nan, "S3": np.nan, 
            "O1a":0.35, "O1b":0.35, 
            "M1": np.nan, "M2": np.nan, "M3":np.nan, "M4":np.nan, "NF":np.nan,
            "M1_5": 0.1, "M1_10": 0.2,  "M1_15": 0.3, "M1_20": 0.4, "M1_25": 0.5, "M1_30": 0.6, 
            "M1_35": 0.7, "M1_40": 0.8, "M1_45": 0.8, "M1_50": 0.8, "M1_55": 0.8, "M1_60": 0.8, 
            "M1_65": 1.0, "M1_70": 1.0, "M1_75": 1.0, "M1_80": 1.0, "M1_85": 1.0, "M1_90": 1.0, "M1_95": 1.0}
    
    # PDF dictionary
    PDFD ={"M3_5": 5,"M3_10": 10,"M3_15": 15,"M3_20": 20,"M3_25": 25,"M3_30": 30,"M3_35": 35,"M3_40": 40,"M3_45": 45,"M3_50": 50,
           "M3_55": 55,"M3_60": 60,"M3_65": 65,"M3_70": 70,"M3_75": 75,"M3_80": 80,"M3_85": 85,"M3_90": 90,"M3_95": 95,"M4_5": 5,
           "M4_10": 10,"M4_15": 15,"M4_20": 20,"M4_25": 25,"M4_30": 30,"M4_35": 35,"M4_40": 40,"M4_45": 45,"M4_50": 50,"M4_55": 55,
           "M4_60": 60,"M4_65": 65,"M4_70": 70,"M4_75": 75,"M4_80": 80,"M4_85": 85,"M4_90": 90,"M4_95": 95,"M3M4_5": 5,"M3M4_10": 10,
           "M3M4_15": 15,"M3M4_20": 20,"M3M4_25": 25,"M3M4_30": 30,"M3M4_35": 35,"M3M4_40": 40,"M3M4_45": 45,"M3M4_50": 50,"M3M4_55": 55,
           "M3M4_60": 60,"M3M4_65": 65,"M3M4_70": 70,"M3M4_75": 75,"M3M4_80": 80,"M3M4_85": 85,"M3M4_90": 90,"M3M4_95": 95}
    
    # PCD dictionary
    PCD = {"M1_5":5,"M1_10":10,"M1_15":15,"M1_20":20,"M1_25":25,"M1_30":30,"M1_35":35,"M1_40":40,"M1_45":45,
           "M1_50":50,"M1_55":55,"M1_60":60,"M1_65":65,"M1_70":70,"M1_75":75,"M1_80":80,"M1_85":85,"M1_90":90,
           "M1_95":95,"M2_5":5,"M2_10":10,"M2_15":15,"M2_20":20,"M2_25":25,"M2_30":30,"M2_35":35,"M2_40":40,
           "M2_45":45,"M2_50":50,"M2_55":55,"M2_60":60,"M2_65":65,"M2_70":70,"M2_75":75,"M2_80":80,"M2_85":85,
           "M2_90":90,"M2_95":95,"M1M2_5":5,"M1M2_10":10,"M1M2_15":15,"M1M2_20":20,"M1M2_25":25,"M1M2_30":30,
           "M1M2_35":35,"M1M2_40":40,"M1M2_45":45,"M1M2_50":50,"M1M2_55":55,"M1M2_60":60,"M1M2_65":65,"M1M2_70":70,
           "M1M2_75":75,"M1M2_80":80,"M1M2_85":85,"M1M2_90":90,"M1M2_95":95}
    
    # Create empty dataframe
    cols = ['fueltype', 'mon', 'jd', 'M', 'jd_min',	'lat', 'lon', 'elev', 'ffmc', 'ws',	'waz', 'bui', 'ps',	'saz', 'pc', 'pdf',	'gfl', 'cur', 'time', 'pattern']
    df_data = pd.DataFrame(columns=cols)
    
    # Mix
    print('Creating combinations...')
    if specific_values is False:
        ws = np.arange(WS[0], WS[1] + 1, WS[2])
        ffmc = np.arange(FFMC[0], FFMC[1] + FFMC[2], FFMC[2])
        bui = np.arange(BUI[0], BUI[1] + BUI[2], BUI[2])
        pss = np.arange(ps[0], ps[1] + ps[2], ps[2])
        sazs = np.arange(saz[0], saz[1] + saz[2], saz[2])
    else:
        ws = WS
        ffmc = FFMC
        bui = BUI
        pss = ps
        sazs = saz
    combinations = np.stack(np.meshgrid(FT, ws, ffmc, bui, pss, sazs), -1).reshape(-1, 6)    
    _aux = pd.DataFrame(combinations)
    _aux.columns = ['fueltype', 'ws', 'ffmc', 'bui', 'ps', 'saz']
    
    # Populating gfl
    for ft in GFLD.keys():
        _aux.loc[_aux['fueltype'] == ft, 'gfl'] = GFLD[ft]
    
    # Populate
    print('Populating final df...')
    df_data  = pd.concat([df_data, _aux], axis=0)
    df_data['lat'] = lat
    df_data['lon'] = lon
    df_data['time'] = time
    
    # Return generated dataset
    return df_data

In [4]:
# Generate the data
df_datagen = data_generator(FT=['C1','C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'D1', 'D2', 'S1', 'S2', 'S3', 'O1a', 'O1b', 'M1', 'M2', 'M3', 'M4'],
                            WS=np.arange(0,80,5),
                            FFMC=[40, 85, 89, 91, 95],
                            BUI=[10, 25, 45, 65, 99],
                            ps=[0, 25, 50, 75, 100],
                            saz=[0, 45, 90, 135, 180],
                            specific_values=True)
print('Dims:', df_datagen.shape)

instance_name = 'DataGenerated_Small'
df_datagen.to_csv('DataGenerated_Small' + '.csv', index=False)
df_datagen.head()

Creating combinations...
Populating final df...
Dims: (180000, 20)


/var/folders/4q/2ngj48zn1b9cn0bd_kz6gg8c0000gn/T/ipykernel_19903/3672600981.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_data  = pd.concat([df_data, _aux], axis=0)


,fueltype,mon,jd,M,jd_min,lat,lon,elev,ffmc,ws,waz,bui,ps,saz,pc,pdf,gfl,cur,time,pattern
0,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,0,NaN,NaN,0.75,NaN,20,NaN
1,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,45,NaN,NaN,0.75,NaN,20,NaN
2,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,90,NaN,NaN,0.75,NaN,20,NaN
3,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,135,NaN,NaN,0.75,NaN,20,NaN
4,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,180,NaN,NaN,0.75,NaN,20,NaN


#### FBP data generator in Python

In [5]:
# Dict class [dict to object]
class Dict2Class(object):
    def __init__(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])

# Run FBP given input data to obtain HROS, BROS, and FROS
def run_fbp(fbp_path, args):
    execArray = [
        '/Users/minho/Documents/GitHub/Cell2Fire/Cell2FireC_Canada/Cell2Fire',
        '--input-data', args['InFolder'] + '.csv',
        # '--nsims', args['NSims'],
        '--verbose' if args['verbose'] else '',
    ]
    
    # Verbose
    if args['verbose']:
        print('ExecArray:', execArray)
    
    # Output Log
    if args['OutFolder'] is not None:
        if os.path.isdir(args['OutFolder']) is False:
            os.makedirs(args['OutFolder'])
    LogName = os.path.join(args['OutFolder'], 'FBPDataGenerator_Output.csv')
        
    # Perform the call 
    with open(LogName, 'w') as output:
        proc = subprocess.Popen(execArray, stdout=output)
        proc.communicate()
    return_code = proc.wait()
    if return_code != 0:
        raise RunTimeError(f'C++ returned {return_code}. \nTry looking at {LogName}.')
        
    # End data generator execution
    print('End of FBP DataGenerator execution...')
    return execArray

In [6]:
# Paths
FBP_DATA_GEN_PATH = '/Users/minho/Documents/Github/Cell2Fire/notebooks'

# Arguments
args = {
    'InFolder': os.path.join(FBP_DATA_GEN_PATH, instance_name),
    'OutFolder': os.path.join(FBP_DATA_GEN_PATH, instance_name),
    'NSims':1,
    'verbose': False,
    'instance': instance_name
}

# Dict to object
args_obj = Dict2Class(args)

# Run Cell2Fire with FBP
execArray = run_fbp(fbp_path=FBP_DATA_GEN_PATH, args=args)

PermissionError: [Errno 13] Permission denied: '/Users/minho/Documents/GitHub/Cell2Fire/Cell2FireC_Canada/Cell2Fire'

In [7]:
# Read
df_ros = pd.read_csv('DataGenerated_Small.csv', low_memory=False)
print('Dims:', df_ros.shape)
df_ros.head()

# df_train_data.to_csv('training_data_small_08202023.csv', index=False)

Dims: (180000, 20)


,fueltype,mon,jd,M,jd_min,lat,lon,elev,ffmc,ws,waz,bui,ps,saz,pc,pdf,gfl,cur,time,pattern
0,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,0,NaN,NaN,0.75,NaN,20,NaN
1,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,45,NaN,NaN,0.75,NaN,20,NaN
2,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,90,NaN,NaN,0.75,NaN,20,NaN
3,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,135,NaN,NaN,0.75,NaN,20,NaN
4,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,NaN,10,0,180,NaN,NaN,0.75,NaN,20,NaN


In [8]:
# Merge inputs/outputs
df_train_data = pd.concat([df_datagen, df_ros], axis=1)
print('Original Dims:', df_train_data.shape)

# # Take away NAN values for ROS (useless)
# df_train_data.dropna(subset=['HROS'], inplace=True)
# df_train_data.dropna(subset=['FROS'], inplace=True)
# df_train_data.dropna(subset=['BROS'], inplace=True)
# print('Dims:', df_train_data.shape)
df_train_data.head()

Original Dims: (180000, 40)


,fueltype,mon,jd,M,jd_min,lat,lon,elev,ffmc,ws,...,waz,bui,ps,saz,pc,pdf,gfl,cur,time,pattern
0,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,...,NaN,10,0,0,NaN,NaN,0.75,NaN,20,NaN
1,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,...,NaN,10,0,45,NaN,NaN,0.75,NaN,20,NaN
2,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,...,NaN,10,0,90,NaN,NaN,0.75,NaN,20,NaN
3,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,...,NaN,10,0,135,NaN,NaN,0.75,NaN,20,NaN
4,C1,NaN,NaN,NaN,NaN,51.621244,-115.608378,NaN,40,0,...,NaN,10,0,180,NaN,NaN,0.75,NaN,20,NaN


In [12]:
import csv
import subprocess

def process_line_with_cpp_executable(input_line):
    """
    Process a single line of input with the C++ executable and return the ROS result.
    """
    # Prepare the command to execute the C++ program
    command = ['/Users/minho/Documents/GitHub/Cell2Fire/Cell2FireC_Canada/FBP3', input_line]
    
    # Run the C++ executable and capture the output
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        output = result.stdout.strip()  # Read the standard output
        return output
    except subprocess.CalledProcessError as e:
        print(f"Error running C++ executable: {e}")
        return None

def read_and_process_csv(input_csv, output_csv):
    """
    Read the input CSV file line by line, process each line with the C++ executable,
    and write the results to the output CSV file.
    """

    ros_results = {'HROS':[], 'BROS':[], 'FROS':[]}

    with open(input_csv, mode='r') as infile, open(output_csv, mode='w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # Write the header for the output CSV
        writer.writerow(['Input', 'Head_ROS', 'Backfire_ROS', 'Flankfire_ROS'])
        
        for row in reader:
            input_line = ','.join(row)
            # print(f"Processing: {input_line}")
            
            # Process the line with the C++ executable
            output = process_line_with_cpp_executable(input_line)
            
            if output is not None:
                # Parse the output into the respective ROS values
                values = output.split(',')
                print(values)
                if len(values) == 3:
                    head_ros, backfire_ros, flankfire_ros = values
                    
                    ros_results['HROS'].append(head_ros)
                    ros_results['BROS'].append(backfire_ros)
                    ros_results['FROS'].append(flankfire_ros)

                else:
                    print(f"Unexpected output format: {output}")
            else:
                print(f"Failed to process line: {input_line}")

            break
    
    return ros_results

if __name__ == "__main__":
    input_csv_file = '/Users/minho/Documents/GitHub/Cell2Fire/notebooks/DataGenerated_Small.csv'
    output_csv_file = '/Users/minho/Documents/GitHub/Cell2Fire/notebooks/DataGenerated_Small_now.csv'
    ros_results = read_and_process_csv(input_csv_file, output_csv_file)


['']
Unexpected output format: 


#### FBP Python version

In [13]:
import math

# Constants
SLOPELIMIT_ISI = 0.01  # Defined according to use in the original code

class Inputs:
    def __init__(self, fueltype, ffmc, ws, gfl, bui, lat, lon, time, pattern, mon, jd, jd_min, waz, ps, saz, pc, pdf, cur, elev, hour, hourly):
        self.fueltype = fueltype
        self.ffmc = ffmc
        self.ws = ws
        self.gfl = gfl
        self.bui = bui
        self.lat = lat
        self.lon = lon
        self.time = time
        self.pattern = pattern
        self.mon = mon
        self.jd = jd
        self.jd_min = jd_min
        self.waz = waz
        self.ps = ps
        self.saz = saz
        self.pc = pc
        self.pdf = pdf
        self.cur = cur
        self.elev = elev
        self.hour = hour
        self.hourly = hourly

class FuelCoefs:
    def __init__(self, fueltype, q, bui0, cbh, cfl, a, b, c):
        self.fueltype = fueltype
        self.q = q
        self.bui0 = bui0
        self.cbh = cbh
        self.cfl = cfl
        self.a = a
        self.b = b
        self.c = c

class MainOuts:
    def __init__(self, hffmc, sfc, csi, rso, fmc, sfi, rss, isi, be, sf, raz, wsv, ff, jd_min, jd, covertype):
        self.hffmc = hffmc
        self.sfc = sfc
        self.csi = csi
        self.rso = rso
        self.fmc = fmc
        self.sfi = sfi
        self.rss = rss
        self.isi = isi
        self.be = be
        self.sf = sf
        self.raz = raz
        self.wsv = wsv
        self.ff = ff
        self.jd_min = jd_min
        self.jd = jd
        self.covertype = covertype

class SndOuts:
    def __init__(self, lb, area, perm, pgr, lbt):
        self.lb = lb
        self.area = area
        self.perm = perm
        self.pgr = pgr
        self.lbt = lbt

def rate_of_spread(inp, ptr, at):
    """
    Calculate the rate of spread based on input parameters and fuel coefficients.

    :param inp: Inputs object with attributes like ffmc, waz, ws, and other parameters.
    :param ptr: Fuel coefficients object.
    :param at: Outputs object where results will be stored.
    :return: Calculated rate of spread (RSS).
    """
    # Calculate ffmc effect
    at.ff = ffmc_effect(inp.ffmc)
    print("FFMC EFFECT : ", at.ff)
    # Set raz (this is directly taken from inp.waz in the C code)
    at.raz = inp.waz
    print("RAZ : ", at.raz)
    # Calculate isz
    isz = 0.208 * at.ff
    print("ISZ : ", isz)
    # Calculate wsv based on ps
    print("PS (SLOPE) : ", inp.ps)
    if inp.ps > 0:
        at.wsv = slope_effect(inp, ptr, at, isz)
    else:
        at.wsv = inp.ws
    print("WSV: ", at.wsv)
    # Calculate fw
    if at.wsv < 40.0:
        fw = math.exp(0.05039 * at.wsv)
    else:
        fw = 12.0 * (1.0 - math.exp(-0.0818 * (at.wsv - 28)))
    print("FW : ", fw)
    # Calculate isi
    at.isi = isz * fw
    print("ISI : ", at.isi)
    # Calculate rsi
    mu = [0.0]  # List to simulate pass-by-reference
    rsi = ros_calc(inp, ptr, at.isi, mu)
    print("RSI : ", rsi)
    # Calculate rss
    at.rss = rsi * bui_effect(ptr, at, inp.bui)
    print("RSS : ", at.rss)
    return at.rss


def ffmc_effect(ffmc):
    mc = 147.2 * (101.0 - ffmc) / (59.5 + ffmc)
    ff = 91.9 * math.exp(-0.1386 * mc) * (1 + mc**5.31 / 49300000.0)
    return ff

def slope_effect(inp, ptr, at, isi):
    ps = min(inp.ps, 70.0)
    at.sf = min(math.exp(3.533 * (ps / 100.0)**1.2), 10.00)
    print("DEBUG SLOPE (at.sf) : ",at.sf)
    isf = 0.0
    
    if ptr.fueltype.startswith("M"):
        isf = ISF_mixedwood(ptr, isi, inp.pc, at.sf) if ptr.fueltype.startswith("M1") or ptr.fueltype.startswith("M2") else ISF_deadfir(ptr, isi, inp.pdf, at.sf)
    else:
        rsz = ros_calc(inp, ptr, isi, [0.0])
        print("DEBUG SLOPE (RSZ) : ",rsz)
        rsf = rsz * at.sf
        print("DEBUG SLOPE (RSF) : ",rsf)
        print("DEBUG SLOPE (RSZ) : ",rsz)
        print("DEBUG SLOPE (RSZ) : ",rsz)
        print("DEBUG SLOPE (RSZ) : ",rsz)
        check = 1.0 - (rsf / (0.0 * ptr.a))**(1.0 / ptr.c) if rsf > 0.0 else 1.0
        check = max(check, SLOPELIMIT_ISI)
        isf = (1.0 / (-1.0 * ptr.b)) * math.log(check)
    
    if isf == 0.0:
        isf = isi
    wse1 = math.log(isf / (0.208 * at.ff)) / 0.05039
    wse = wse1 if wse1 <= 40.0 else min(28.0 - math.log(1.0 - isf / (2.496 * at.ff)) / 0.0818, 0.999 * 2.496 * at.ff)
    
    wrad = inp.waz / 180.0 * math.pi
    wsx = inp.ws * math.sin(wrad)
    wsy = inp.ws * math.cos(wrad)
    srad = inp.saz / 180.0 * math.pi
    wsex = wse * math.sin(srad)
    wsey = wse * math.cos(srad)
    wsvx = wsx + wsex
    wsvy = wsy + wsey
    wsv = math.sqrt(wsvx**2 + wsvy**2)
    raz = math.acos(wsvy / wsv) * 180.0 / math.pi
    if wsvx < 0:
        raz = 360 - raz
    at.raz = raz
    
    print("DEBUG SLOPE (wsv) : ",wsv)

    return float(wsv)

def ISF_mixedwood(ptr, isz, pc, sf):
    rsf_c2 = sf * ptr.a * (1.0 - math.exp(-ptr.b * isz))**ptr.c
    check = 1.0 - (rsf_c2 / ptr.a)**(1.0 / ptr.c) if rsf_c2 > 0.0 else 1.0
    check = max(check, SLOPELIMIT_ISI)
    isf_c2 = (1.0 / (-1.0 * ptr.b)) * math.log(check)
    
    mult = 0.2 if ptr.fueltype.startswith("M2") else 1.0
    for i in range(numfuels):
        if ptr.fueltype.startswith("D1"):
            break
        ptr += 1
    
    rsf_d1 = sf * (mult * ptr.a) * (1.0 - math.exp(-ptr.b * isz))**ptr.c
    check = 1.0 - (rsf_d1 / (mult * ptr.a))**(1.0 / ptr.c) if rsf_d1 > 0.0 else 1.0
    check = max(check, SLOPELIMIT_ISI)
    isf_d1 = (1.0 / (-1.0 * ptr.b)) * math.log(check)
    
    return float(pc / 100.0) * isf_c2 + float(100 - pc) / 100.0 * isf_d1

def ISF_deadfir(ptr, isz, pdf, sf):
    rsf_max = sf * ptr.a * (1.0 - math.exp(-ptr.b * isz))**ptr.c
    check = 1.0 - (rsf_max / ptr.a)**(1.0 / ptr.c) if rsf_max > 0.0 else 1.0
    check = max(check, SLOPELIMIT_ISI)
    isf_max = (1.0 / (-1.0 * ptr.b)) * math.log(check)
    
    mult = 0.2 if ptr.fueltype.startswith("M4") else 1.0
    
    for i in range(numfuels):
        if ptr.fueltype.startswith("D1"):
            break
        ptr += 1
    
    rsf_d1 = sf * (mult * ptr.a) * (1.0 - math.exp(-ptr.b * isz))**ptr.c
    check = 1.0 - (rsf_d1 / (mult * ptr.a))**(1.0 / ptr.c) if rsf_d1 > 0.0 else 1.0
    check = max(check, SLOPELIMIT_ISI)
    isf_d1 = (1.0 / (-1.0 * ptr.b)) * math.log(check)
    
    return float(pdf / 100.0) * isf_max + float(100.0 - pdf) / 100.0 * isf_d1

def ros_calc(inp, ptr, isi, mult):

    print("ROS CALC (Fuel type) : ", inp.fueltype)

    if inp.fueltype.startswith("O1"):
        return grass(ptr, inp.cur, isi, mult)
    elif inp.fueltype.startswith("M"):
        return mixed_wood(ptr, isi, mult, inp.pc) if inp.fueltype.startswith("M1") or inp.fueltype.startswith("M2") else dead_fir(ptr, inp.pdf, isi, mult)
    elif inp.fueltype.startswith("D2"):
        return D2_ROS(ptr, isi, inp.bui, mult)
    
    return conifer(ptr, isi, mult)

def grass(ptr, cur, isi, mult):
    mu = 0.176 + 0.02 * (cur - 58.8) if cur >= 58.8 else 0.005 * (math.exp(0.061 * cur) - 1.0)
    ros = mu * (ptr.a * (1.0 - math.exp(-ptr.b * isi))**ptr.c)
    mu = max(mu, 0.001)
    mult[0] = mu
    return ros

def mixed_wood(ptr, isi, mu, pc):
    mu[0] = pc / 100.0
    ros_c2 = ptr.a * (1.0 - math.exp(-ptr.b * isi))**ptr.c
    mult = 0.2 if ptr.fueltype.startswith("M2") else 1.0
    
    for i in range(numfuels):
        if ptr.fueltype.startswith("D1"):
            break
        ptr += 1
    
    ros_d1 = ptr.a * (1.0 - math.exp(-ptr.b * isi))**ptr.c
    ros = (pc / 100.0) * ros_c2 + mult * (100 - pc) / 100.0 * ros_d1
    return ros

def dead_fir(ptr, pdf, isi, mu):
    greenness = 0.2 if ptr.fueltype.startswith("M4") else 1.0
    ros_max = ptr.a * (1.0 - math.exp(-ptr.b * isi))**ptr.c
    
    for i in range(numfuels):
        if ptr.fueltype.startswith("D1"):
            break
        ptr += 1
    
    ros_d1 = ptr.a * (1.0 - math.exp(-ptr.b * isi))**ptr.c
    ros = (pdf / 100.0) * ros_max + (100.0 - pdf) / 100.0 * greenness * ros_d1
    mu[0] = pdf / 100.0
    return ros

def D2_ROS(ptr, isi, bui, mu):
    mu[0] = 1.0
    return ptr.a * (1.0 - math.exp(-ptr.b * isi))**ptr.c if bui >= 80 else 0.0

def bui_effect(ptr, at, bui):
    bui_avg = 50.0
    if bui == 0:
        bui = 1.0
    at.be = math.exp(bui_avg * math.log(ptr.q) * ((1.0 / bui) - (1.0 / ptr.bui0)))
    return at.be

def backfire_ros(inp, ptr, at, bisi):
    mult = [0.0]
    bros = ros_calc(inp, ptr, bisi, mult)
    bros *= bui_effect(ptr, at, inp.bui)
    return bros

def flankfire_ros(ros, bros, lb):
    return (ros + bros) / (lb * 2.0)

def conifer(ptr, isi, mu):
    mu = 1.0
    ros = ptr.a * pow((1.0 - math.exp(-ptr.b * isi)), ptr.c)
    print("CONIFER ROS : ", ros)
    return ros

def l_to_b(ft, ws):

    if ft.startswith("O1"):
        return 1.0 if ws < 1.0 else (1.1 * math.pow(ws, 0.464))
    else:
        return 1.0 + 8.729 * math.pow(1.0 - math.exp(-0.030 * ws), 2.155)

def backfire_isi(at):
    bfw = math.exp(-0.05039 * at.wsv)
    return 0.208 * at.ff * bfw


# Placeholder for numfuels, should be set accordingly
numfuels = 10


In [21]:
# Example usage
df = pd.read_csv('/Users/minho/Documents/GitHub/Cell2Fire/notebooks/DataGenerated_Small.csv')

# Extract the first row as a dictionary
data = df.iloc[0].to_dict()

inputs = Inputs(
    fueltype=data.get('fueltype', ''),
    ffmc=data.get('ffmc', 0.0),
    ws=data.get('ws', 0.0),
    gfl=data.get('gfl', 0.0),
    bui=data.get('bui', 0.0),
    lat=data.get('lat', 0.0),
    lon=data.get('lon', 0.0),
    time=data.get('time', ''),
    pattern=data.get('pattern', ''),
    mon=data.get('mon', 0),
    jd=data.get('jd', 0.0),
    jd_min=data.get('jd_min', 0.0),
    waz=data.get('waz', 0.0),
    ps=data.get('ps', 0.0),
    saz=data.get('saz', 0.0),
    pc=data.get('pc', 0.0),
    pdf=data.get('pdf', 0.0),
    cur=data.get('cur', 0.0),
    elev=data.get('elev', 0.0),
    hour=data.get('hour', 0.0),
    hourly=data.get('hourly', '')
)


In [22]:
class FuelCoefs:
    def __init__(self, fueltype, a, b, c, q, bui0, cbh, cfl):
        self.fueltype = fueltype
        self.a = a
        self.b = b
        self.c = c
        self.q = q
        self.bui0 = bui0
        self.cbh = cbh
        self.cfl = cfl

def setup_fuel_coefs():
    fuel_coefs_list = []

    # Define all fuel types and their coefficients
    fuel_data = [
        ("M1", 110.0, 0.0282, 1.5, 0.80, 50, 6, 0.80),
        ("M2", 110.0, 0.0282, 1.5, 0.80, 50, 6, 0.80),
        ("M3", 120.0, 0.0572, 1.4, 0.80, 50, 6, 0.80),
        ("M4", 100.0, 0.0404, 1.48, 0.80, 50, 6, 0.80),
        ("C1", 90.0, 0.0649, 4.5, 0.90, 72, 2, 0.75),
        ("C2", 110.0, 0.0282, 1.5, 0.70, 64, 3, 0.80),
        ("C3", 110.0, 0.0444, 3.0, 0.75, 62, 8, 1.15),
        ("C4", 110.0, 0.0293, 1.5, 0.80, 66, 4, 1.20),
        ("C5", 30.0, 0.0697, 4.0, 0.80, 56, 18, 1.20),
        ("C6", 30.0, 0.0800, 3.0, 0.80, 62, 7, 1.80),
        ("C7", 45.0, 0.0305, 2.0, 0.85, 106, 10, 0.50),
        ("D1", 30.0, 0.0232, 1.6, 0.90, 32, 0, 0.0),
        ("S1", 75.0, 0.0297, 1.3, 0.75, 38, 0, 0.0),
        ("S2", 40.0, 0.0438, 1.7, 0.75, 63, 0, 0.0),
        ("S3", 55.0, 0.0829, 3.2, 0.75, 31, 0, 0.0),
        ("O1a", 190.0, 0.0310, 1.40, 1.000, 1, 0, 0.0),
        ("O1b", 250.0, 0.0350, 1.7, 1.000, 1, 0, 0.0),
        ("D2", 6.0, 0.0232, 1.6, 0.90, 32, 0, 0.0),
    ]

    # Populate the list with FuelCoefs objects
    for fueltype, a, b, c, q, bui0, cbh, cfl in fuel_data:
        fuel_coefs_list.append(FuelCoefs(fueltype, a, b, c, q, bui0, cbh, cfl))

    return fuel_coefs_list

def get_fuel_coefs(fueltype, fuel_coefs_list):
    for coefs in fuel_coefs_list:
        if coefs.fueltype == fueltype:
            return coefs
    raise ValueError(f"Fuel type {fueltype} not found")

# Initialize fuel coefficients
fuel_coefs_list = setup_fuel_coefs()

In [23]:
fuel_coefs = get_fuel_coefs(inputs.fueltype, fuel_coefs_list)

fuel_coefs

In [24]:
def compute_rates_of_spread(inputs):
    fuel_coefs = get_fuel_coefs(inputs.fueltype, fuel_coefs_list)
    
    # Placeholder for MainOuts; initialize with appropriate values
    at = MainOuts(
        hffmc=ffmc_effect(inputs.ffmc),
        sfc=0.0, csi=0.0, rso=0.0,
        fmc=inputs.ffmc, sfi=0.0, rss=0.0,
        isi=0.0, be=0.0, sf=0.0, raz=0.0,
        wsv=0.0, ff=0.0, jd_min=inputs.jd_min,
        jd=inputs.jd, covertype=""
    )
    
    mult = [0.0]
    
    # Compute ROS
    print("COMPUTING HEAD ROS")
    ros = rate_of_spread(inputs, fuel_coefs, at)
    
    # Compute BROS
    bisi = backfire_isi(at)
    bros = backfire_ros(inputs, fuel_coefs, at, bisi)
    
    # Compute FROS
    lb = l_to_b(inputs.fueltype, inputs.ws)
    fros = flankfire_ros(ros, bros, lb)
    
    return ros, bros, fros

# Example usage
ros, bros, fros = compute_rates_of_spread(inputs)
print(f"Rate of Spread: {ros}")
print(f"Back Rate of Spread: {bros}")
print(f"Flank Rate of Spread: {fros}")


COMPUTING HEAD ROS
FFMC EFFECT :  0.16694583594823537
RAZ :  nan
ISZ :  0.034724733877232954
PS (SLOPE) :  0
WSV:  0
FW :  1.0
ISI :  0.034724733877232954
ROS CALC (Fuel type) :  C1
CONIFER ROS :  1.0965212638333807e-10
RSI :  1.0965212638333807e-10
RSS :  6.966355614127019e-11
ROS CALC (Fuel type) :  C1
CONIFER ROS :  1.0965212638333807e-10
Rate of Spread: 6.966355614127019e-11
Back Rate of Spread: 6.966355614127019e-11
Flank Rate of Spread: 6.966355614127019e-11


In [25]:
fuel_coefs = get_fuel_coefs(inputs.fueltype, fuel_coefs_list)

# Placeholder for MainOuts; initialize with appropriate values
at = MainOuts(
    hffmc=ffmc_effect(inputs.ffmc),
    sfc=0.0, csi=0.0, rso=0.0,
    fmc=inputs.ffmc, sfi=0.0, rss=0.0,
    isi=0.0, be=0.0, sf=0.0, raz=0.0,
    wsv=0.0, ff=0.0, jd_min=inputs.jd_min,
    jd=inputs.jd, covertype=""
)

mult = [0.0]

# Compute ROS
ros = ros_calc(inputs, fuel_coefs, at.isi, mult)

# Compute BROS
bisi = 0.0  # Example; replace with actual value or calculation
bros = backfire_ros(inputs, fuel_coefs, at, bisi)

# Compute FROS
lb = l_to_b(inputs.fueltype, inputs.ws)
fros = flankfire_ros(ros, bros, lb)

ROS CALC (Fuel type) :  C1
CONIFER ROS :  0.0
ROS CALC (Fuel type) :  C1
CONIFER ROS :  0.0


#### KITRAL (CHILE)
- Contenido Humedad: 0.5% steps  -> 20 + 1 vals
- Humedad: 1% steps -> 30 + 1 vals
- Viento: 0.5 km/hr steps -> 60 + 1 vals
- Pendiente: 0.5% steps -> 60 + 1 vals

FCH ("factor contenido humedad"), FMC ("factor de propagación"), FP ("factor pendiente") and FV ("factor viento")
- FP and FV represents the effect of slope and wind speed.

In [26]:
# Constants
speed = [0.0188800,0.0160270,0.0102350,0.0086900,0.0010090,0.0076030,0.0081470,0.0016720,0.0048860,0.0103210,0.0092340,0.0017870,0.0043420,0.0022490,0.0014410,0.0009790,0.0015560,0.0023650,0.0131740,0.0059730,0.0024810,0.0027120,0.0065160,0.0032550,0.0025960,0.0097770,0.0054290,0.0037990,0.0013250,0.0021340,0.0019030]
carga = [0.684,0.527,0.918,0.617,0.649,2.923,1.910,3.308,1.383,3.029,3.529,3.189,1.903,2.624,2.310,3.544,2.164,1.954,0.838,3.019,3.333,3.249,4.087,3.714,4.063,0.905,3.164,2.742,2.464,8.250,7.125]
heat = [3928,3928,3928,3928,3800,4693,4693,4572,4572,5000,5087,4500,4500,4600,4550,4452,4452,4452,4399,4870,4870,4870,4870,4870,4870,4372,4816,4816,4684,4746,4652]

speed_dict = dict(zip(np.arange(1,32,1),speed))
carga_dict = dict(zip(np.arange(1,32,1),carga))
heat_dict = dict(zip(np.arange(1,32,1),heat))

# Variables
fm = np.arange(1,32,1).tolist()
ch = np.arange(0,21,5).tolist()
# humidity = np.arange(0,32,2).tolist()
wind = np.arange(0,61,5).tolist()
slope = np.arange(0,61,5).tolist()

In [27]:
# Iterate through all lists
a = [fm, ch, wind, slope]
aa = list(itertools.product(*a))

# Convert lists to DF
adf = pd.DataFrame(np.asarray(aa))
adf.columns = ['nftype','ch','wind','slope']

# Map constants based on nftype with variables
adf['speed'] = adf['nftype'].map(speed_dict)
adf['carga'] = adf['nftype'].map(carga_dict)
adf['heat'] = adf['nftype'].map(heat_dict)
# adf['fmc'] = adf.speed

# Variables
adf['fch'] = [(389.1624-14.3*ch+0.02*ch**2)/(3.559+1.6615*ch+2.6239*ch**2) for ch in adf.ch]
adf['fp'] = [1+0.023322*slope+0.00013585*slope**2 for slope in adf.slope]
adf['fv'] = [1+0.51218*wind-0.007151*wind**2 for wind in adf.wind]

# Outputs
l1 = 2.233/1.411
l2 = -0.01031/0.01745
adf['HROS'] = adf.speed * adf.fch * (adf.fp + adf.fv)
adf['lb'] = [1.0 + pow(l1 * np.exp(-l2 * ws) - l1, 2.0) for ws in adf.wind]
adf['hb'] = [(lb + np.sqrt(pow(lb,2) - 1.0)) /  (lb - np.sqrt(pow(lb, 2) - 1.0)) for lb in adf.lb]

adf['BROS'] = adf.HROS/adf.hb
adf['FROS'] = (adf.HROS + adf.BROS) / ( adf.lb * 2.0)

adf = adf[['nftype', 'speed', 'carga', 'heat', 'ch','wind','slope','fch','fp','fv','HROS','BROS','FROS']]
# adf.to_csv("data/kitral_training_3ros_full.csv",index=None)

/var/folders/4q/2ngj48zn1b9cn0bd_kz6gg8c0000gn/T/ipykernel_19903/3493328234.py:25: RuntimeWarning: divide by zero encountered in scalar divide
  adf['hb'] = [(lb + np.sqrt(pow(lb,2) - 1.0)) /  (lb - np.sqrt(pow(lb, 2) - 1.0)) for lb in adf.lb]
